# Tools

## Github

#### Clone a repository based on url

In [ ]:
from typing import Optional, Dict, Any
from git import Repo, GitCommandError
from langchain.tools import tool
import os, shutil


@tool("git_clone")
def git_clone_tool(
    repo_url: str,
    dest: str,
    branch: Optional[str] = None,
    overwrite: bool = False,
) -> Dict[str, Any]:
    """
    Clone a Git repository into ./repositories/{dest} using GitPython.

    Args:
        repo_url: HTTPS or SSH URL of the repository.
        dest: Name of the destination folder for the clone inside ./repositories/.
        branch: Optional branch to check out.
        overwrite: If True, overwrite existing destination folder.
    Returns:
        A dict with success (bool), dest (str), and error/stdout messages.
    """

    try:
        # Ensure repositories/ root exists
        root_dir = os.path.join(os.getcwd(), "repositories")
        os.makedirs(root_dir, exist_ok=True)

        # Full destination path inside repositories/
        full_dest = os.path.join(root_dir, dest)

        # Handle overwrite
        if os.path.exists(full_dest):
            if overwrite:
                shutil.rmtree(full_dest)
            else:
                return {"success": False, "error": f"Destination {full_dest} already exists."}


        # Clone options
        kwargs = {}
        if branch:
            kwargs["branch"] = branch
            full_dest = f"{full_dest}/{branch}"

        os.makedirs(full_dest, exist_ok=True)
        repo = Repo.clone_from(repo_url, full_dest, **kwargs)

        return {
            "success": True,
            "dest": full_dest,
            "branch": repo.active_branch.name if not repo.head.is_detached else "detached",
            "error": None,
        }
    except GitCommandError as e:
        return {"success": False, "dest": dest, "error": str(e)}
    except Exception as e:
        return {"success": False, "dest": dest, "error": str(e)}

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    "openai:gpt-5-nano",
    tools=[git_clone_tool],
    prompt="Act as an assistant. Use the git_clone_tool to download a repository.",
)

result = agent.invoke({"messages": [HumanMessage("Can you clone the following github repository: https://github.com/simonskodt/arch-reconstruct-ai, feel free to overwrite if a clone already exists, ")]})

#### Add Github MCP server based on url

In [ ]:
from typing import Dict, Any
from langchain.tools import tool
from experiments.utils.mcp_client_factory import load_mcp_config, save_mcp_config, create_mcp_client_from_config

@tool("add_mcp_server")
def add_mcp_server_tool(
    name: str,
    url: str,
    transport: str = "streamable_http",
) -> Dict[str, Any]:
    """
    Add a new MCP server to the configuration.
    
    Args:
        name: Name identifier for the MCP server
        url: URL of the MCP server
        transport: Transport type (default: "streamable_http")
    Returns:
        Dict with success status and current config

    Note: 
        MCP tools accessed via clients are not hot reloaded or dynamically  updated,
        a new agent or tool instance has to be  
    """
    try:
        # Load existing config
        config = load_mcp_config()
        
        # Add new server
        config[name] = {
            "url": url,
            "transport": transport
        }
    
        save_mcp_config(config)
        
        return {
            "success": True,
            "message": f"Added MCP server '{name}'",
            "config": config
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "config": {}
        }
    
@tool("add_github_repository_as_mcp_server")
def add_github_repository_as_mcp_tool(repo_url: str, server_name: str) -> Dict[str, Any]:
    """
    Add a GitHub repository as an MCP server using gitmcp.io.
    
    Args:
        repo_url: GitHub repository URL (e.g., https://github.com/owner/repo)
        server_name: Name identifier for the MCP server
    Returns:
        Dict with success status and current config
    """
    # Extract the repository path from the GitHub URL
    if "github.com/" in repo_url:
        # Extract everything after github.com/
        repo_path = repo_url.split("github.com/", 1)[1]
        # Remove .git suffix if present
        if repo_path.endswith(".git"):
            repo_path = repo_path[:-4]
        gitmcp_url = f"https://gitmcp.io/{repo_path}"
    else:
        raise ValueError("Invalid GitHub repository URL")
    
    tool_input = {
        "name": server_name,
        "url": gitmcp_url,
    }

    return add_mcp_server_tool.invoke(tool_input)

    

@tool("remove_mcp_server")
def remove_mcp_server_tool(
    name: str,
) -> Dict[str, Any]:
    """
    Remove an MCP server from the configuration.
    
    Args:
        name: Name identifier of the MCP server to remove
    Returns:
        Dict with success status and current config

    Note: 
        MCP tools accessed via clients are not hot reloaded or dynamically  updated,
        a new agent or tool instance has to be 
    """
    try:
        config = load_mcp_config()
        
        if name not in config:
            return {
                "success": False,
                "error": f"MCP server '{name}' not found",
                "config": config
            }
        
        del config[name]
        save_mcp_config(config)
        
        return {
            "success": True,
            "message": f"Removed MCP server '{name}'",
            "config": config
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "config": {}
        }

@tool("list_mcp_servers")
def list_mcp_servers() -> Dict[str, Any]:
    """
    List all configured MCP servers.
    
    Returns:
        Dict with success status and list of servers
    """
    try:
        config = load_mcp_config()
        return {
            "success": True,
            "servers": list(config.keys()),
            "config": config
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "config": {}
        }

In [ ]:
from langchain_core.messages import HumanMessage

import sys
import os
# Add the parent directory to sys.path so 'experiments' can be imported
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
from experiments.utils.agent_factory import create_agent_with_valid_tools

tools = [add_github_repository_as_mcp_tool, add_mcp_server_tool, remove_mcp_server_tool, list_mcp_servers, git_clone_tool]

agent = create_agent_with_valid_tools(
    "openai:gpt-5-nano",
    tools=tools,
    prompt="""Act as an assistant.
                When using tools:
                - Use tools if relevant before answering.
            """
)


stream = agent.astream({"messages": [HumanMessage("Can you clone the github repository: https://github.com/simonskodt/arch-reconstruct-ai")]})
async for chunk in stream:
    print(chunk)


result = await agent.ainvoke({"messages": [HumanMessage("Can you list the MCP servers available")]})
print(result)

result = await agent.ainvoke({"messages": [HumanMessage("Can you take the following github repository: https://github.com/simonskodt/arch-reconstruct-ai, and make it into a MCP server")]})
print(result)




client = create_mcp_client_from_config()
mcp_tools = await client.get_tools() 
tools += mcp_tools

[print(tool.name) for tool in tools]

agent = create_agent_with_valid_tools(
    "openai:gpt-5-nano",
    tools=tools, # Tools cannot be dynamically  or hot reloaded?, agent has to be recreated  
    prompt="""Act as an assistant.
                When using tools:
                - Use tools if relevant before answering.
            """
)
result = await agent.ainvoke({"messages": [HumanMessage("Can you list the MCP servers available")]})
print(result)

